In [1]:
import json
from tqdm import tqdm

In [2]:
big = "memoryalpha"
small = "stexpanded"
top_k = 10

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/matched_nodes_communities_leiden/top_10_pairs/" + big + "-" + small + "_top_10_pairs.json"

communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

small_communities = []
gold_pairs = []
matched_communities = []

In [3]:
with open(communities_small_file) as csf, open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf:
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [4]:
found = 0
all_pairs = len(gold_pairs)

for gold_pair in tqdm(gold_pairs):
    for matched_community in matched_communities:
        node = matched_community[0]
        community = set()
        for i in range(0, top_k):
            community.update(small_communities[matched_community[1][i]])
        if gold_pair[0] == node and gold_pair[1] in community:
            found += 1
            # print(matched_community[0], matched_community[1], gold_pair)
            break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

100%|██████████| 1779/1779 [19:13<00:00,  1.54it/s]

Gold pairs in matched communities top 10: 56.7734682405846%


In [5]:
found_small = 0
for gold_pair in gold_pairs:
    for small_community in small_communities:
        if gold_pair[1] in small_community:
            found_small += 1
            break

print("Small: " + str(found_small / all_pairs * 100) + "%")

Small: 100.0%
